In [1]:
import numpy as np
import pandas as pd

# Dropping Columns with the DataFrame Object 

In [2]:
!ls

Data Cleaning Tutorial - Real Python.ipynb
Datasets
follow_tutorial.ipynb


In [3]:
bookData = pd.read_csv('Datasets/BL-Flickr-Images-Book.csv')
bookData.head() #.head() shows first couple of rows of data, for initial assessment 

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [4]:
#we see a couple of unwanted columns for our particular task, so lets drop them.

#list containing all columns not relating to the books themselves
dropList = ['Edition Statement', 'Publisher', 'Contributors','Corporate Author','Corporate Contributors', 'Former owner', 'Engraver','Issuance type', 'Flickr URL', 'Shelfmarks']

#inplace as True and axis=1 tells pandas that we want changes made directly to the object 
bookData.drop(dropList, inplace=True, axis=1)

bookData.head()

,Identifier,Place of Publication,Date of Publication,Title,Author
0,206,London,1879 [1878],Walter Forbes. [A novel.] By A. A,A. A.
1,216,London; Virtue & Yorston,1868,All for Greed. [A novel. The dedication signed...,"A., A. A."
2,218,London,1869,Love the Avenger. By the author of “All for Gr...,"A., A. A."
3,472,London,1851,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S."
4,480,London,1857,"[The World in which I live, and my place in it...","A., E. S."


In [ ]:
#another way to drop the columns is to pass the list directly to columns, this omits the axis=1 and is more intutive 

bookData.drop(columns=dropList, inplace=True)

bookData.head()

# Changing index of a data frame

In [5]:
# many times, it is useful to have a unique identifier column to be the index of your data

# being able to change this index improves labeling and slicing of our dataset

# we can check if a column is filled with unique values with: 

bookData['Identifier'].is_unique

True

In [6]:

# now that we have confirmed the uniqueness of the column, let's set the index to that column

bookData = bookData.set_index('Identifier')  
# we do this because by default .set_index() returns a modified version of the data frame
# to do it without this reassignment step we can use:
# bookData.set_index('Identifier', inplace=True)

bookData.head()

,Place of Publication,Date of Publication,Title,Author
Identifier,,,,
206,London,1879 [1878],Walter Forbes. [A novel.] By A. A,A. A.
216,London; Virtue & Yorston,1868,All for Greed. [A novel. The dedication signed...,"A., A. A."
218,London,1869,Love the Avenger. By the author of “All for Gr...,"A., A. A."
472,London,1851,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S."
480,London,1857,"[The World in which I live, and my place in it...","A., E. S."


In [7]:
# now we should be able to select rows by identifier using .loc 

bookData.loc[206]

Place of Publication                               London
Date of Publication                           1879 [1878]
Title                   Walter Forbes. [A novel.] By A. A
Author                                              A. A.
Name: 206, dtype: object

## so previously we had our indeces from 0 like a python list, but now our indeces are values of 'Identifier'

# now let us tidy up some data in the columns

In [8]:
# see how many columns are "dtype: object" which essentially are strs, aka a mess of characters

bookData.dtypes.value_counts()

object    4
dtype: int64

In [9]:
# lets enforce numeric properties onto the publish date column

#check from id 1905, just the Date of Publication column
bookData.loc[1905:, 'Date of Publication'].head()

Identifier
1905           1888
1929    1839, 38-54
2836           1897
2854           1865
2956        1860-63
Name: Date of Publication, dtype: object

## this looks very messy, as one book should only have one date of publication 

In [10]:
# lets clean this up: 

# 1. remove extra dates in square brackets
# 2. remove uncertain dates and replace with NaN value from numpy
# 3. convert date ranges to start date
# 4. convert nan strings to NaN counter part

# this is when regex comes in handy 
# we just need to find any 4 digits at the start of a string

# the r in front tells python it is a raw string (backslash is no longer an escape character), \d means digit, 
# {4} means repeat 4 times, and () groups \d and {4} to create 4 digits, ^ means search from the start

regex = r'^(\d{4})'

extracted = bookData['Date of Publication'].str.extract(regex, expand=False)

extracted.head()

Identifier
206    1879
216    1868
218    1869
472    1851
480    1857
Name: Date of Publication, dtype: object

In [11]:
# so now we see much nicer strings in the Date of Publication column ( which we saved into extracted, original
# is still messy as seen below )

bookData['Date of Publication'].head()

Identifier
206    1879 [1878]
216           1868
218           1869
472           1851
480           1857
Name: Date of Publication, dtype: object

In [12]:
# great, now lets change all the data types in extracted to float 

extracted = pd.to_numeric(extracted)
extracted.head()

Identifier
206    1879.0
216    1868.0
218    1869.0
472    1851.0
480    1857.0
Name: Date of Publication, dtype: float64

In [13]:

# now that everything is good, lets replace the Date of Publication column values with those from extracted

bookData['Date of Publication'] = extracted

bookData['Date of Publication'].dtype

dtype('float64')

In [14]:

# now we are able to do numeric operations on our column (below operation shows that 10% of data is null values):

bookData['Date of Publication'].isnull().sum() / len(bookData)

0.11717147339205986

In [15]:
# average Date of publication: 

bookData['Date of Publication'].sum() / len(bookData)

1639.0848316640522


# we can combine str methods and numpy to clean columns more effectively

In [16]:
# data frame object have the .str attribute. This allows for string operations in pandas to mimic native python 
# string methods like .split() and .capitalize()

# np.where(condition, [x,y]) returns either x or y, depending on condition
# this can also be nested to create a compound if, then statement allowing us to computer for multiple conditions
# lets use this to clean the Place of Publication column. 

# first lets see the contents of the Place of Publication column. 

bookData['Place of Publication'].head()

Identifier
206                      London
216    London; Virtue & Yorston
218                      London
472                      London
480                      London
Name: Place of Publication, dtype: object

In [17]:
# the tutorial showcases two specific cases: 

bookData.loc[4157862]

Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                  1867
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Name: 4157862, dtype: object

In [18]:
bookData.loc[4159587]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1834
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Name: 4159587, dtype: object

In [19]:
# these two books have the same Place of Publication, but one has hypens while one does not. 
# we can clean these up with str.contains() to create a boolean map of which rows have a certain value

regex2 = r'Newcastle.upon.Tyne'

pub = bookData['Place of Publication']

newCastle = pub.str.contains(regex2)

# as we can see, the lines containing the words have the boolean map of True
print(newCastle[4159587])
print(newCastle[4157862])

newCastle[:5]

True
True


Identifier
206    False
216    False
218    False
472    False
480    False
Name: Place of Publication, dtype: bool

In [20]:
# lets create some more conditions
regex3 = 'London'

london = pub.str.contains(regex3)


# we combine them with np.where() here I nested them to fix London, Newcastle, and all hypen issues 

bookData['Place of Publication'] = np.where(newCastle, 'Newcastle upon Tyne', 
                                            np.where(london, 'London', pub.str.replace('_', ' ')))

print(bookData.loc[4157862])
bookData['Place of Publication'].head()

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1867
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Name: 4157862, dtype: object


Identifier
206    London
216    London
218    London
472    London
480    London
Name: Place of Publication, dtype: object

In [21]:

# now the rows are a lot cleaner for Place of Publication
bookData.head()

,Place of Publication,Date of Publication,Title,Author
Identifier,,,,
206,London,1879.0,Walter Forbes. [A novel.] By A. A,A. A.
216,London,1868.0,All for Greed. [A novel. The dedication signed...,"A., A. A."
218,London,1869.0,Love the Avenger. By the author of “All for Gr...,"A., A. A."
472,London,1851.0,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S."
480,London,1857.0,"[The World in which I live, and my place in it...","A., E. S."
